In [1]:
import numpy as np
import opt_einsum as oe
import scipy

import quimb as qu
import quimb.tensor as qtn

In [2]:
from tenso import *

### stuff needed to create tensors

In [3]:
# in numpy arrays
def make_Ux(N, beta_p, dtype = np.complex128):
    """Return as MPO the U_x evolution operator at time-parameter beta_p."""

    tb = np.array( [[np.cos(beta_p), 1j*np.sin(beta_p)],[1j*np.sin(beta_p), np.cos(beta_p)]], dtype=dtype)
    
    arrays = [ np.expand_dims(tb, axis=0) ] + \
             [ np.expand_dims(tb, axis=(0,1)) for _ in range(N-2) ] + \
             [ np.expand_dims(tb, axis=0) ]

    return arrays

def Wz(N, Uk : np.array, xi : int, marginal = False, dtype = np.complex128):
    """The tensors of Eq. 17 of reference paper."""
    
    bond_dim = len(Uk)

    shape = (bond_dim,2,2) if marginal else (bond_dim,bond_dim,2,2)
    tensor = np.zeros( shape, dtype = dtype )

    coeff = np.power( Uk/np.sqrt(N+1), 1/N)
    exx = 1j * np.arange(bond_dim) * np.pi / (N + 1)    # check: N+1

    for kk in range(bond_dim):
        spin_matrix = np.diag(
            [ coeff[kk]*np.exp(exx[kk]*(1-xi)), 
              coeff[kk]*np.exp(exx[kk]*(1+xi)) ] 
        )
        if marginal:  tensor[kk,:,:] = spin_matrix
        else:         tensor[kk,kk,:,:] = spin_matrix
    
    return tensor

def make_Uz(N : int, Uk , xi ): #, bond_dim = None, dtype = np.complex128):
    bond_dim = None
    dtype = np.complex128
    """Return as MPO the U_z evolution operator at time s_p (defined indirectly by Uk)."""

    # Uk must be a vector for all k values, while p is fixed 
    # xi must be a single sample from dataset
    
    assert len(xi) == N, 'not matching dims'

    arrays = [ Wz(N, Uk, xi[0], marginal = True, dtype = dtype) ] + \
             [ Wz(N, Uk, xi[i+1], dtype = dtype) for i in range(N-2) ] + \
             [ Wz(N, Uk, xi[N-1], marginal = True, dtype = dtype) ]

    return arrays

In [4]:
# in Quimb...

def make_Ux_tn(N, beta_p, dtype = np.complex128):
    """Return as MPO the U_x evolution operator at time-parameter beta_p."""

    tb = np.array( [[np.cos(beta_p), 1j*np.sin(beta_p)],[1j*np.sin(beta_p), np.cos(beta_p)]], dtype=dtype)
    
    arrays = [ np.expand_dims(tb, axis=0) ] + \
             [ np.expand_dims(tb, axis=(0,1)) for _ in range(N-2) ] + \
             [ np.expand_dims(tb, axis=0) ]

    return qtn.tensor_1d.MatrixProductOperator( arrays )

def make_Uz_tn(N : int, Uk , xi ): #, bond_dim = None, dtype = np.complex128):
    bond_dim = None
    dtype = np.complex128
    """Return as MPO the U_z evolution operator at time s_p (defined indirectly by Uk)."""

    # Uk must be a vector for all k values, while p is fixed 
    # xi must be a single sample from dataset
    
    assert len(xi) == N, 'not matching dims'

    arrays = [ Wz(N, Uk, xi[0], marginal = True, dtype = dtype) ] + \
             [ Wz(N, Uk, xi[i+1], dtype = dtype) for i in range(N-2) ] + \
             [ Wz(N, Uk, xi[N-1], marginal = True, dtype = dtype) ]

    return qtn.tensor_1d.MatrixProductOperator( arrays )

In [5]:
N = 4

Uk_FT = np.ones((N+1,10), dtype=np.complex128)
xi = np.loadtxt('dataxi.txt')[:,0:N]

In [6]:
def contract_mps_mpo_margin(state_site, operator_site):
    return oe.contract('ax,byx->aby', state_site, operator_site).reshape( (-1, state_site.shape[1]) )

#def contract_mps_mpo(state_site, operator_site):
#    raise Exception('wrooong! use the other one')
#    return oe.contract('abc,ijcl->abijl', state_site, operator_site).reshape( (state_site.shape[0]*operator_site.shape[0], state_site.shape[1]*operator_site.shape[1], state_site.shape[2]) )

def contract_mps_mpo(state_site, operator_site):
    cc = oe.contract('abc,ijlc->aibjl', state_site, operator_site).reshape( (state_site.shape[0]*operator_site.shape[0], state_site.shape[1]*operator_site.shape[1], operator_site.shape[3]) )
    return cc


def multiply(mps, mpo):
    assert len(mps) == len(mpo), 'must have same number of sites'
    assert len(mps) >= 2

    out = []

    out.append( contract_mps_mpo_margin(mps[0], mpo[0]) )

    for i in range(N-2):
        out.append(
            contract_mps_mpo(mps[i+1], mpo[i+1])
        )
    
    out.append( contract_mps_mpo_margin(mps[-1], mpo[-1]) )

    return out


#a = oe.contract('ab,ibk->aik', psi[0], ux[0]).reshape( (psi[0].shape[0]*ux[0].shape[0], psi[0].shape[1]) )
#b = oe.contract('ab,ibk->aik', psi[1], ux[1]).reshape( (psi[1].shape[0]*ux[1].shape[0], psi[1].shape[1]) )

#psia = oe.contract('aik,aij->kj', a, b)

In [7]:
#TN_ux = make_Ux_tn(N, 0.1)
#
#TN_psi = qu.tensor.tensor_builder.MPS_product_state( 
#    [ np.array([[2**-0.5, 2**-0.5]], dtype=np.complex128) ] * N,
#    tags=['psi'],
#    #site_ind_id='s{}'
#)
#
#TN_ux.apply(TN_psi)

In [8]:
# compute with Quimb ---

TN_psi = qu.tensor.tensor_builder.MPS_product_state( 
    [ np.array([[2**-0.5, 2**-0.5]], dtype=np.complex128) ] * N, tags=['psi'],
)
TN_uz = make_Uz_tn(N, Uk_FT[:,0], xi[0])

quimb_psi_1 = TN_uz.apply(TN_psi)
quimb_psi_2 = TN_uz.apply(quimb_psi_1)
quimb_psi_3 = TN_uz.apply(quimb_psi_2)

In [9]:
# compute with my functions ---

psi = [ np.array([[2**-0.5, 2**-0.5]], dtype=np.complex128) ] + [ np.array([[[2**-0.5, 2**-0.5]]], dtype=np.complex128) for i in range(N-2) ] + [ np.array([[2**-0.5, 2**-0.5]], dtype=np.complex128) ]
uz = make_Uz(N, Uk_FT[:,0], xi[0])

my_psi_1 = multiply(psi, uz)
my_psi_2 = multiply(my_psi_1, uz)
my_psi_3 = multiply(my_psi_2, uz)

In [10]:
# operators ok
np.allclose( uz[1] , np.array( TN_uz.tensors[1].data ) )

True

In [11]:
np.allclose( my_psi_1[1] , np.array( quimb_psi_1.tensors[1].data ) )

True

In [12]:
np.allclose( my_psi_2[1] , np.array( quimb_psi_2.tensors[1].data ) )

True

In [13]:
np.sum( np.abs(my_psi_2[1] - np.array( quimb_psi_2.tensors[1].data ) ) )

6.591336035270332e-17

In [14]:
np.allclose( my_psi_3[0] , np.array( quimb_psi_3.tensors[0].data ) )

True

In [15]:
np.sum( np.abs(my_psi_3[0] - np.array( quimb_psi_3.tensors[0].data ) ) )

1.080083700143892e-15

## advanced test with random values

In [16]:
def make_rand_mpo(N : int, bond_dim = 1 ): #, bond_dim = None, dtype = np.complex128):
    dtype = np.complex128
    """Return a random MPO."""

    shape_margin = (bond_dim,2,2)
    shape_middle = (bond_dim,bond_dim,2,2)

    arrays = [ np.random.uniform(low=-1.0, size=shape_margin) + 1.j * np.random.uniform(low=-1.0, size=shape_margin) ] + \
             [ np.random.uniform(low=-1.0, size=shape_middle) + 1.j * np.random.uniform(low=-1.0, size=shape_middle) for _ in range(N-2)] + \
             [ np.random.uniform(low=-1.0, size=shape_margin) + 1.j * np.random.uniform(low=-1.0, size=shape_margin) ]

    #arrays = [ np.random.uniform(low=-1.0, size=shape_margin) ] + \
    #         [ np.random.uniform(low=-1.0, size=shape_middle) for _ in range(N-2)] + \
    #         [ np.random.uniform(low=-1.0, size=shape_margin) ]

    return arrays

In [17]:
# compute with my functions
dty = np.float64

psi = [ np.array([[2**-0.5, 2**-0.5]], dtype=dty) ] + [ np.array([[[2**-0.5, 2**-0.5]]], dtype=dty) for i in range(N-2) ] + [ np.array([[2**-0.5, 2**-0.5]], dtype=dty) ]
uz = make_rand_mpo(N, bond_dim=5)

my_psi_1 = multiply(psi, uz)
my_psi_2 = multiply(my_psi_1, uz)
my_psi_3 = multiply(my_psi_2, uz)

In [18]:
# compute with Quimb

TN_psi = qu.tensor.tensor_builder.MPS_product_state( 
    [ np.array([[2**-0.5, 2**-0.5]], dtype=dty) ] * N, tags=['psi'],
)
TN_uz = qtn.tensor_1d.MatrixProductOperator( uz )

quimb_psi_1 = TN_uz.apply(TN_psi)
quimb_psi_2 = TN_uz.apply(quimb_psi_1)
quimb_psi_3 = TN_uz.apply(quimb_psi_2)

In [19]:
TN_uz.L

4

In [20]:
# operators ok
assert np.allclose( uz[1] , np.array( TN_uz.tensors[1].data ) ), 'operators are not equal'

In [21]:
np.allclose( my_psi_1[1] , np.array(quimb_psi_1.tensors[1].data )  )

True

In [22]:
np.allclose( my_psi_2[1] , np.array( quimb_psi_2.tensors[1].data ) )

True

In [23]:
np.allclose( my_psi_3[1] , np.array( quimb_psi_3.tensors[1].data ) )

True

In [24]:
my_psi_3[0].shape

(125, 2)

In [25]:
# https://tensornetwork.readthedocs.io/en/latest/stubs/tensornetwork.FiniteMPS.html#tensornetwork.FiniteMPS

# canonicalization

In [26]:
#  # pylint: disable=arguments-differ
#def canonicalize(mps, normalize: bool = True) -> np.number:
#    """Bring the MPS into canonical form according to
#    `center_position`. If `center_position` is `None`, the
#    MPS is canonicalized with `center_position = 0`.
#    Args:
#      normalize: If `True`, normalize matrices when shifting the
#        orthogonality center.
#    Returns:
#      `Tensor`: The norm of the MPS.
#    """
#    N = len(mps)
#    if self.center_position is not None:
#      pos = self.center_position
#      if pos >= N // 2:
#        self.center_position = 0
#        self.position(N - 1, normalize=normalize)
#      else:
#        self.center_position = len(mps) - 1
#        self.position(0, normalize=normalize)
#      return self.position(pos, normalize=normalize)
#
#    self.center_position = len(mps) - 1
#    return self.position(0, normalize=normalize)

In [27]:
#psi = [ np.array([[2**-0.5, 2**-0.5]], dtype=dty) ] + [ np.array([[[2**-0.5, 2**-0.5]]], dtype=dty) for i in range(N-2) ] + [ np.array([[2**-0.5, 2**-0.5]], dtype=dty) ]
#canonicalize(psi, 2)

In [28]:
import tensornetwork as tn

rank = 5
phys_dim = 2
bond_dim = 3

# build the mps:
# the state is canonically normalized when we define the class FiniteMPS
mpstate = tn.FiniteMPS.random(
  d = [phys_dim for _ in range(rank)],
  D = [bond_dim for _ in range(rank-1)],
  dtype = np.float32,
  canonicalize=True
)
print( mpstate.center_position )
#mpstate.tensors[-3:]

0


In [29]:
mpstate.tensors[-1].shape

(2, 2, 1)

In [30]:
def from_lib_to_me(mpsg):
    N = len(mpstate.tensors)

    array = [ mpstate.tensors[0][0,:,:].copy() ] + \
            [ np.swapaxes(mpstate.tensors[i+1], 1,2).copy() for i in range(N-2)] + \
            [ mpstate.tensors[-1][:,:,0].copy() ]
    return array

mpstate.center_position = rank - 1 
mpstate.canonicalize(normalize = True)
from_lib_to_me(mpstate.tensors)[-3:]

[array([[[-0.4323081 ,  0.88470507],
         [-0.8174609 , -0.35909432],
         [-0.19019341, -0.03837419]],
 
        [[ 0.08641699,  0.07071481],
         [ 0.11192763, -0.02078479],
         [-0.5552282 ,  0.4591189 ]],
 
        [[-0.1339117 ,  0.00241945],
         [ 0.32257357, -0.2929086 ],
         [ 0.255728  ,  0.6147264 ]]], dtype=float32),
 array([[[-0.54842025, -0.29229578],
         [-0.561151  , -0.559218  ]],
 
        [[-0.646773  ,  0.44187936],
         [ 0.5546917 , -0.2540994 ]],
 
        [[-0.00957291, -0.01157812],
         [ 0.003635  ,  0.01105511]]], dtype=float32),
 array([[-0.50390667, -0.3868852 ],
        [-0.69231844,  0.34218866]], dtype=float32)]

In [31]:
mpstate.center_position = 1
mpstate.canonicalize(normalize = True)
from_lib_to_me(mpstate.tensors)[-3:]

[array([[[-3.5158476e-01,  8.0489606e-01],
         [ 4.3767715e-01,  1.9226307e-01],
         [-1.9951833e-03, -4.0255624e-04]],
 
        [[ 6.3861328e-01,  5.4511029e-01],
         [-5.3090543e-01,  9.3883052e-02],
         [-5.0829776e-02,  4.2066615e-02]],
 
        [[ 5.5152082e-01,  4.5062743e-02],
         [ 6.0930860e-01, -5.6736958e-01],
         [-1.5742149e-02, -1.9522827e-02]]], dtype=float32),
 array([[[-0.62977403, -0.43910578],
         [-0.45386535, -0.4523019 ]],
 
        [[ 0.48120353, -0.3433202 ],
         [-0.71242887,  0.3781792 ]],
 
        [[-0.48013577, -0.3555188 ],
         [ 0.25126928,  0.7615378 ]]], dtype=float32),
 array([[-0.793183  , -0.60898334],
        [-0.60898334,  0.793183  ]], dtype=float32)]

In [32]:
# take the state (non canonized) from Google TN library in my format
mpme = from_lib_to_me(mpstate)
mpme[-3:]

[array([[[-3.5158476e-01,  8.0489606e-01],
         [ 4.3767715e-01,  1.9226307e-01],
         [-1.9951833e-03, -4.0255624e-04]],
 
        [[ 6.3861328e-01,  5.4511029e-01],
         [-5.3090543e-01,  9.3883052e-02],
         [-5.0829776e-02,  4.2066615e-02]],
 
        [[ 5.5152082e-01,  4.5062743e-02],
         [ 6.0930860e-01, -5.6736958e-01],
         [-1.5742149e-02, -1.9522827e-02]]], dtype=float32),
 array([[[-0.62977403, -0.43910578],
         [-0.45386535, -0.4523019 ]],
 
        [[ 0.48120353, -0.3433202 ],
         [-0.71242887,  0.3781792 ]],
 
        [[-0.48013577, -0.3555188 ],
         [ 0.25126928,  0.7615378 ]]], dtype=float32),
 array([[-0.793183  , -0.60898334],
        [-0.60898334,  0.793183  ]], dtype=float32)]

In [33]:
for e in mpme:
    print(e.shape)

(2, 2)
(2, 3, 2)
(3, 3, 2)
(3, 2, 2)
(2, 2)


In [34]:
mpstate.center_position = 3
mpstate.canonicalize(normalize = True)
from_lib_to_me(mpstate.tensors) # this is the 'solution'

[array([[-0.86879534, -0.4951713 ],
        [-0.4951713 ,  0.86879534]], dtype=float32),
 array([[[-0.930658  , -0.3517344 ],
         [-0.08769027, -0.01767108],
         [-0.24792081,  0.74421257]],
 
        [[-0.00218697,  0.10076643],
         [-0.4633883 , -0.8816289 ],
         [-0.5449467 ,  0.29616895]]], dtype=float32),
 array([[[-0.39513645,  0.9008857 ],
         [-0.83247435, -0.3209054 ],
         [ 0.18794647,  0.02949039]],
 
        [[-0.04344174, -0.06876154],
         [-0.19518676,  0.08807521],
         [-0.4730875 ,  0.5979005 ]],
 
        [[-0.15964547, -0.01304405],
         [ 0.28721663, -0.2750595 ],
         [-0.3882215 , -0.48144957]]], dtype=float32),
 array([[[-6.4941806e-01, -5.0719672e-01],
         [-1.5145548e-01, -1.8565795e-01]],
 
        [[-7.3600151e-02,  8.0140188e-02],
         [ 4.3512991e-01, -2.4964239e-01]],
 
        [[ 3.3546700e-03,  8.6979751e-05],
         [-4.3031634e-03, -8.4615815e-03]]], dtype=float32),
 array([[-0.97405595, -0.2263

In [35]:
for ss in mpstate.tensors:
    print('*sol', np.linalg.norm(ss) )

*sol 1.4142135
*sol 1.7320508
*sol 1.7320508
*sol 1.0
*sol 1.4142137


In [36]:
mpstate.tensors[-2].shape

(3, 2, 2)

In [37]:
res = canonicalize(mpme, mpstate.center_position, True)

In [38]:
res

[array([[-0.86879534, -0.4951713 ],
        [-0.4951713 ,  0.86879534]], dtype=float32),
 array([[[-0.930658  , -0.3517344 ],
         [-0.08769027, -0.01767108],
         [-0.24792081,  0.74421257]],
 
        [[-0.00218697,  0.10076643],
         [-0.4633883 , -0.8816289 ],
         [-0.5449467 ,  0.29616895]]], dtype=float32),
 array([[[-0.39513645,  0.9008857 ],
         [-0.83247435, -0.3209054 ],
         [ 0.18794647,  0.02949039]],
 
        [[-0.04344174, -0.06876154],
         [-0.19518676,  0.08807521],
         [-0.4730875 ,  0.5979005 ]],
 
        [[-0.15964547, -0.01304405],
         [ 0.28721663, -0.2750595 ],
         [-0.3882215 , -0.48144957]]], dtype=float32),
 array([[[-0.5285904 , -0.38496   ],
         [-0.40654004, -0.3788445 ]],
 
        [[-0.24701199,  0.1762336 ],
         [ 0.36570492, -0.1941275 ]],
 
        [[ 0.0048343 ,  0.00357958],
         [-0.00252993, -0.00766763]]], dtype=float32),
 array([[-0.793183  , -0.60898334],
        [-0.60898334,  0.7931